In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('data_immo.raw.csv',delimiter='\t')

In [3]:
df.head()

,index0,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,0,yes,2,1986,3,2016,2016_03,up floor,3,108333.33333333333,3915.6626506024095,325000,83,85221
1,1,yes,NAN,1972,3,2016,2016_03,up floor,3,115000.0,4107.142857142857,345000,84,85521
2,2,yes,2,1968,3,2016,2016_03,up floor,3,123333.33333333333,4805.194805194805,370000,77,81369
3,3,yes,2,1963,3,2016,2016_03,up floor,3,133000.0,4244.68085106383,399000,94,82110
4,4,yes,2,1973,3,2016,2016_03,up floor,3,122933.33333333333,4789.61038961039,368800,77,81735


In [4]:
df.dtypes

index0                int64
terrace              object
bedrooms             object
construction_year    object
date_month            int64
date_year             int64
date_year_month      object
floor                object
rooms                object
price_room           object
price_surface        object
price                object
surface              object
zip                   int64
dtype: object

In [5]:
df.isna().sum()

index0               0
terrace              0
bedrooms             0
construction_year    0
date_month           0
date_year            0
date_year_month      0
floor                0
rooms                0
price_room           0
price_surface        0
price                0
surface              0
zip                  0
dtype: int64

In [6]:
df.replace('NAN',np.NaN,inplace=True)

In [7]:
df.isna().sum()

index0                 0
terrace                0
bedrooms             319
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                113
rooms                 34
price_room            35
price_surface        212
price                  1
surface              211
zip                    0
dtype: int64

In [8]:
(df.isnull().sum() / len(df))*100

index0                0.000000
terrace               0.000000
bedrooms             36.708861
construction_year     5.638665
date_month            0.000000
date_year             0.000000
date_year_month       0.000000
floor                13.003452
rooms                 3.912543
price_room            4.027618
price_surface        24.395857
price                 0.115075
surface              24.280783
zip                   0.000000
dtype: float64

In [9]:
df[df['price'].isna()].T

,367
index0,4
terrace,yes
bedrooms,NaN
construction_year,2016
date_month,8
date_year,2016
date_year_month,2016_08
floor,NaN
rooms,5
price_room,NaN


In [10]:
df.drop(df[df['price'].isna()].index,inplace = True)

In [11]:
df.isna().sum()

index0                 0
terrace                0
bedrooms             318
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                112
rooms                 34
price_room            34
price_surface        211
price                  0
surface              211
zip                    0
dtype: int64

In [12]:
df.drop(['index0'],axis=1,inplace=True)
df.dtypes

terrace              object
bedrooms             object
construction_year    object
date_month            int64
date_year             int64
date_year_month      object
floor                object
rooms                object
price_room           object
price_surface        object
price                object
surface              object
zip                   int64
dtype: object

In [13]:
for col in list(df.columns):
    try:
        df[col] = df[col].astype(int)
    except:
        df[col] = df[col].astype(str)

In [14]:
for col in list(df.columns):
    if df[col].dtypes == object:
        try:
            df[col] = df[col].astype(float)
        except:
            df[col] = df[col].astype(str)

In [15]:
df.dtypes 

terrace               object
bedrooms             float64
construction_year    float64
date_month             int32
date_year              int32
date_year_month        int32
floor                 object
rooms                float64
price_room           float64
price_surface        float64
price                  int32
surface              float64
zip                    int32
dtype: object

In [16]:
df['floor'].value_counts()

up floor        687
nan             112
ground floor     69
Name: floor, dtype: int64

In [19]:
def fill10pct():
    if np.random.randint(0,9) == 0:
        return 'ground floor'
    else:
        return 'up floor'

In [20]:
for index, value in df.iterrows():
    if df.loc[index,'floor']=='nan':
        df.loc[index,'floor'] = fill10pct()

In [21]:
df['floor'].value_counts()

up floor        787
ground floor     81
Name: floor, dtype: int64

In [22]:
conditions =[
    (df['rooms'].isnull()) & (df['bedrooms'].isnull()) & (df['surface'].isnull()),
    (df['rooms'].isnull()) & (df['bedrooms'].isnull()) & (df['surface']>75),
    (df['rooms'].isnull()) & (df['bedrooms'].notnull()),
]
choices = [2,3,df['bedrooms']+1]
df['rooms'] = np.select(conditions,choices,default=2)

In [23]:
df['rooms'].value_counts()

2.0    836
3.0     24
4.0      8
Name: rooms, dtype: int64

In [24]:
df['bedrooms'] = np.where(df['bedrooms'].isnull(), df['rooms']-1, df['bedrooms'])

In [25]:
df.isnull().sum()

terrace                0
bedrooms               0
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                  0
rooms                  0
price_room            34
price_surface        211
price                  0
surface              211
zip                    0
dtype: int64

In [27]:
df.groupby(['rooms'])['surface'].mean()

rooms
2.0     89.988924
3.0     91.100000
4.0    100.400000
Name: surface, dtype: float64

In [29]:
conditions = [
    (df['rooms']==2 & df['surface'].isnull()),
    (df['rooms']==3 & df['surface'].isnull()),
    (df['rooms']==4 & df['surface'].isnull())
]
choices = [90,91.1,100]
df['surface'] = np.select(conditions,choices,default=90.5)

In [31]:
df['construction_year'].fillna(df['construction_year'].mean(),inplace=True)

In [32]:
df.isnull().sum()

terrace                0
bedrooms               0
construction_year      0
date_month             0
date_year              0
date_year_month        0
floor                  0
rooms                  0
price_room            34
price_surface        211
price                  0
surface                0
zip                    0
dtype: int64

In [33]:
df['price_room'] = df['price'] / df['rooms']
df['price_surface'] = df['price'] / df['surface']

In [34]:
df.isnull().sum()

terrace              0
bedrooms             0
construction_year    0
date_month           0
date_year            0
date_year_month      0
floor                0
rooms                0
price_room           0
price_surface        0
price                0
surface              0
zip                  0
dtype: int64

In [35]:
df.to_csv('data_immo.clean.csv',sep='\t',index=False)